# Adversarial Outlier Exposure

from the paper Adversarial Outlier Exposure, presented at SKILL GI 2023 in Berlin by Thomas Botschen (OVGU Magdeburg)

In [7]:
import numpy as np
import time
import torch
from omegaconf import OmegaConf
from tqdm import tqdm
from torch.utils.data import Dataset
from pytorch_ood.loss import OutlierExposureLoss
from attacks import fgsm_softmax, pgd_softmax, mifgsm_softmax
from eval import create_metrics_dataset
from utils import  get_training_datasets, get_model, check_for_epsilon_order

### edit the config file to configure experiments as you need

In [8]:
args = OmegaConf.load("config.yaml")

In [9]:
args

{'model': 'wrn', 'dataset_in': 'cifar10', 'dataset_out': 'GAN_IMG', 'epochs': 1, 'learning_rate': 0.001, 'batch_size': 256, 'oe_batch_size': 256, 'test_bs': 256, 'momentum': 0.9, 'decay': 0.0005, 'score': 'softmax', 'adv_oe': 'FGSM', 'eps_oe': 0.07, 'alpha_oe': 0.03, 'steps_oe': 20, 'epsilon_order': False}

In [10]:
def generate_adversarial_outlier(args, model, inlier, outlier, target_inlier, target_outlier, eps_oe, device):
    attack_methods = {
        "None": lambda *args, **kwargs: outlier,
        "FGSM": fgsm_softmax,
        "PGD": pgd_softmax,
        "MIFGSM": mifgsm_softmax
    }

    if args.score == "softmax" and args.adv_oe in attack_methods:
        attack_method = attack_methods[args.adv_oe]
        common_kwargs = {
            "model": model,
            "inlier": inlier,
            "outlier": outlier,
            "target_inlier": target_inlier,
            "target_outlier": target_outlier,
            "eps_oe": eps_oe,
            "device": device
        }
        if args.adv_oe in ["PGD", "MIFGSM"]:
            common_kwargs.update({
                "alpha_oe": args.alpha_oe,
                "steps_oe": args.steps_oe
            })
        return attack_method(**common_kwargs)
    else:
        return None


def get_device():
    if torch.cuda.is_available():
        print("CUDA is available. Using GPU for computations.")
        return torch.device('cuda')
    else:
        print("CUDA is not available. Using CPU for computations.")
        return torch.device('cpu')


def create_data_loader(data, batch_size, shuffle, num_workers=0, pin_memory=True):
    return torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=shuffle,
                                       num_workers=num_workers, pin_memory=pin_memory)


def prepare_data_loaders(args):
    # Retrieve logic data
    train_data_in, train_data_out = get_training_datasets(args)

    # Define data loaders
    train_loader_in = create_data_loader(train_data_in, args.batch_size, shuffle=True)
    train_loader_out = create_data_loader(train_data_out, args.oe_batch_size, shuffle=False)

    return train_loader_in, train_loader_out


def create_optimizer_and_scheduler(model, args, train_loader_in):
    # define logic parameters
    def cosine_annealing(step, total_steps, lr_max, lr_min):
        return lr_min + (lr_max - lr_min) * 0.5 * (
                1 + np.cos(step / total_steps * np.pi))

    optimizer = torch.optim.SGD(
        model.parameters(), args.learning_rate, momentum=args.momentum,
        weight_decay=args.decay, nesterov=True)

    scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer,
        lr_lambda=lambda step: cosine_annealing(
            step,
            args.epochs * len(train_loader_in),
            1,  # since lr_lambda computes multiplicative factor
            1e-6 / args.learning_rate))

    return optimizer, scheduler

### Training

In [11]:
def run(args):
    torch.manual_seed(1)
    np.random.seed(1)

    # GPU acceleration
    device = get_device()
    print("Using device:", device)

    model = get_model(args)

    # define data loaders
    train_loader_inlier, train_loader_outlier = prepare_data_loaders(args)

    # define optimizer and scheduler
    optimizer, scheduler = create_optimizer_and_scheduler(model, args, train_loader_inlier)

    criterion = OutlierExposureLoss()

    def train_adversarial_default(epoch):
        loss_avg = 0.0
        model.train()  # enter train mode
        print("Starting adversarial training epoch %d" % epoch)
        print("Length of train_loader_inlier: ", len(train_loader_inlier))
        print("Length of train_loader_outlier: ", len(train_loader_outlier))

        for in_set, out_set in tqdm(zip(train_loader_inlier, train_loader_outlier), total=len(train_loader_inlier)):

            # define variables
            inlier = in_set[0]
            outlier = out_set[0]
            target_inlier = in_set[1]
            target_outlier = out_set[1]

            # stop if we have covered the entire outlier dataset and we have remains (10000%64=16 -> the code would crash)
            if len(outlier) < args.oe_batch_size:
                break

            # check for custom epsilon order
            eps_oe = check_for_epsilon_order(epoch, args)

            # perform adversarial attack on outliers
            perturbed_outlier = generate_adversarial_outlier(args, model, inlier, outlier, target_inlier,
                                                             target_outlier, eps_oe, device)

            # concatenate inlier and perturbed outlier
            data = torch.cat((inlier.to(device), perturbed_outlier.to(device)), 0).to(device)

            # forward
            data = data.float()
            x = model(data)

            # backward
            optimizer.zero_grad()

            # calculate loss
            loss = criterion(x, torch.cat((target_inlier.to(device), target_outlier.to(device)), 0))

            # update parameters
            loss.backward()
            optimizer.step()

            scheduler.step()
            # exponential moving average
            loss_avg = loss_avg * 0.8 + float(loss) * 0.2
        train_loss = loss_avg

        return train_loss

    def run_training():
        training_start = time.time()
        losses = []

        if args.dataset_in in ["cifar10", "cifar100"]:
            for epoch in range(args.epochs):
                begin_epoch = time.time()

                train_loss_epoch = train_adversarial_default(epoch)
                losses.append({"epoch": epoch, "loss": round(train_loss_epoch, 4)})

                print(
                    f'Epoch {epoch + 1:3d} | Time {int(time.time() - begin_epoch):5d} | Train Loss {train_loss_epoch:.4f}')

        else:
            raise ValueError("Unknown dataset: {}".format(args.dataset_in))

        time_needed = time.time() - training_start
        print(f"Finished training in {time_needed}")


        aurocs = create_metrics_dataset(model=model, dataset_to_test=args.dataset_in)

        print(aurocs)


    # now run the program :)
    run_training()

In [12]:
run(args)

CUDA is available. Using GPU for computations.
Using device: cuda
Files already downloaded and verified
Starting adversarial training epoch 0
Length of train_loader_inlier:  196
Length of train_loader_outlier:  196


 99%|█████████▉| 195/196 [02:43<00:00,  1.19it/s]


Epoch   1 | Time   163 | Train Loss 0.6264
Finished training in 163.48838782310486
Files already downloaded and verified
Evaluating textures...


100%|██████████| 123/123 [00:49<00:00,  2.46it/s]


Evaluating lsun resize...


...

Evaluating gaussian noise...


100%|██████████| 83/83 [00:08<00:00, 10.30it/s]


Evaluating uniform noise...


100%|██████████| 83/83 [00:06<00:00, 12.51it/s]

[{'dataset': 'textures', 'metrics': {'AUROC': 0.8869585394859314, 'AUPR-IN': 0.803619921207428, 'AUPR-OUT': 0.9354285001754761, 'FPR95TPR': 0.3813000023365021}}, {'dataset': 'lsun resize', 'metrics': {'AUROC': 0.9252873659133911, 'AUPR-IN': 0.9017477631568909, 'AUPR-OUT': 0.9421669244766235, 'FPR95TPR': 0.22669999301433563}}, {'dataset': 'lsun crop', 'metrics': {'AUROC': 0.9566106796264648, 'AUPR-IN': 0.9464101791381836, 'AUPR-OUT': 0.9658747911453247, 'FPR95TPR': 0.14949999749660492}}, {'dataset': 'tinyimagenet crop', 'metrics': {'AUROC': 0.9426779747009277, 'AUPR-IN': 0.9304371476173401, 'AUPR-OUT': 0.9537640810012817, 'FPR95TPR': 0.1981000006198883}}, {'dataset': 'tinyimagenet resize', 'metrics': {'AUROC': 0.9093412160873413, 'AUPR-IN': 0.8827979564666748, 'AUPR-OUT': 0.9282353520393372, 'FPR95TPR': 0.2734000086784363}}, {'dataset': 'gaussian noise', 'metrics': {'AUROC': 0.9388073682785034, 'AUPR-IN': 0.3222408890724182, 'AUPR-OUT': 0.9968499541282654, 'FPR95TPR': 0.1542000025510788